In [1]:
import numpy as np
import pandas as pd
import spacy

In [2]:
import fr_core_news_md

nlp = fr_core_news_md.load()

In [3]:
def seg_doc(text):
    #V2 : regrouper les enfants isolés de la racine avec la racine

    doc = nlp(text)                      #pour traiter un ensemble de documents, utiliser docs =nlp.pipe(DOCS)

    
    segments = []
    roots = [token for token in doc if token.head == token]
    for root in roots: 
        seg_root = []
        for token in root.lefts:                  #segmenter en sous-arbre
            if len(list(token.subtree)) == 1:
                seg_root.append(token)
            else: 
                temp = []
                for descendant in token.subtree:            
                    if descendant.text == ",":        #segmenter en virgule
                        temp.append(descendant)
                        segments.append(temp)
                        temp = []
                    else:
                        temp.append(descendant)
                if len(temp) != 0:
                    segments.append(temp)
        position_root = len(segments)            
        seg_root.append(root)
        
        for token in root.rights:
            if len(list(token.subtree)) == 1:
                seg_root.append(token)
            else: 
                temp = []
                for descendant in token.subtree:            
                    if descendant.text == ",":        
                        temp.append(descendant)
                        segments.append(temp)
                        temp = []
                    else:
                        temp.append(descendant)
                if len(temp) != 0:
                    segments.append(temp)
        segments.insert(position_root, seg_root)            
#        segments.append(seg_root)

    return segments

def has_verb(doc):
    # déterminer si un segment contient un verbe
    has_verb = False
    for token in doc:
        if token.pos_ in ["VERB", "AUX"]:
            has_verb = True
            break
    return has_verb

def seg2sent(seg):
    # convertir un segment de token en un string de phrase
    phrase = ""
    for token in seg:
        phrase += token.text + " " 
    return phrase

def seg_doc_2(segments):
    #version flatten
    
    temp = [seg for seg in segments]
    for i,seg in enumerate(segments): 
        if has_verb(seg):                       # resegmenter les segments verbaux
            new_segs = seg_doc(seg2sent(seg))
            temp[i] = new_segs
        else:
            temp[i] = [segments[i]]
    return [seg for segs in temp for seg in segs]

def seg_auto(TEXT):
    #resegmenter les segments verbaux tant que le nombre de segments augmente
    segments = seg_doc(TEXT)
    c = 0
    while len(seg_doc_2(segments)) > len (segments):   
        segments = seg_doc_2(segments)
        c += 1
        print(f"segmenter {c} fois")
    return segments

In [4]:
df = pd.read_csv("/Users/rubing/Documents/Donnees/covax_content_sample_2000_medium_sized.csv").drop("Unnamed: 0", axis=1)

In [5]:
TEXT=df.extracted_text[0]
print(TEXT)

Un million de tests pourraient être réalisés d'ici la fin de la semaine. Et un traitement pourrait être disponible d'ici l'été ou l'automne, selon Mike Pence.
Aux États-Unis aussi, les morts se multiplient. Avec quatre nouveaux décès enregistrés lundi, six personnes ont succombé au coronavirus outre-Atlantique, et la barre des cent contaminés a été atteinte. Selon le New York Times , l'administration Trump a alors annoncé qu' un million de tests pourraient être réalisés d'ici la fin de la semaine. Des compagnies privées et des laboratoires académiques ont été poussés à développer et valider leur propre dépistage.  
LIRE AUSSI >> Coronavirus : les Américains sans couverture santé dans l'impasse  
D'autre part, un traitement médical contre le nouveau coronavirus pourrait être disponible "d'ici l'été ou le début de l'automne", a affirmé lundi le vice-président américain Mike Pence , qui coordonne la lutte contre l'épidémie aux États-Unis.  Offre limitée. 2 mois pour 1€ sans engagement Je 

In [6]:
seg_auto(TEXT)

segmenter 1 fois
segmenter 2 fois
segmenter 3 fois
segmenter 4 fois
segmenter 5 fois
segmenter 6 fois


[[Un, million, de, tests],
 [pourraient, .],
 [être, réalisés],
 [d, ', ici, la, fin, de, la, semaine],
 [un, traitement],
 [Et, pourrait],
 [être, disponible],
 [d, ', ', automne, ,],
 [ici, l],
 [été, '],
 [ou, l],
 [selon, Mike, Pence],
 [., ],
 [Aux, États-Unis],
 [les, morts],
 [aussi, ,, se, multiplient, .],
 [Avec, quatre, nouveaux, décès],
 [enregistrés, lundi],
 [six, personnes],
 [,, ont, succombé, ,, .],
 [au, coronavirus, outre-Atlantique],
 [et, la, barre, des, cent, contaminés],
 [a, été, atteinte],
 [Selon, le, New, York, Times],
 [l', administration, Trump],
 [,, a, alors, annoncé, .],
 [un, million, de, tests],
 [qu, ', pourraient],
 [être, réalisés],
 [d, ', ici, la, fin, de, la, semaine],
 [Des, compagnies, privées, et, des, laboratoires, académiques],
 [ont, été, poussés],
 [à, développer],
 [et, valider],
 [leur, propre, dépistage],
 [.,  ],
 [LIRE, AUSSI],
 [>, >],
 [Coronavirus, :],
 [les, Américains, sans, couverture, santé, dans, l', impasse,  ],
 [D', autre, p